In [1]:
import numpy as np
from collections import defaultdict, namedtuple
from text_generator import TextGenerator
import gzip
import cPickle as pickle
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sb
% matplotlib inline
import matplotlib.pyplot as plt
import operator

In [2]:
sb.set_context('poster', rc={'figure.figsize':(12, 12)})

In [3]:
class BiGramModel(object):


    def __init__(self, transcripts, target_period=3, max_vocab_size=None, min_count=50, verbose=True):

        # Initiate some necessary variables
        self._transcripts = transcripts
        self._vocab = defaultdict(dict)
        self._features = list()
        self._targets = list()
        self.max_vocab_size = max_vocab_size
        self.min_count = min_count
        self.stop_words = "a an the for which our by as we that on in with or is are also and of to you these " \
                          "from at last where will i now how have be per during about these this - was year " \
                          "were call it has us than so not like very but million quarter over new first " \
                          "third their would those there . ? ! , & % $ [ ] ( ) ; and/or your 's 're 'd 've 'll".split()

        # Set the target period
        assert target_period in (3, 30, 60, 90), 'Invalid target period: {} days'.format(target_period)
        self.target_period = target_period

        # Build the vocabulary
        self.build_vocab(verbose)
        if self.max_vocab_size:
            self.finalize_vocab()
        self.TFIDF()


    def build_vocab(self, verbose):

        for key, transcript in self._transcripts.iteritems():

            abnormal_return = self.get_return(transcript)

            if abnormal_return < -0.05:
                self._targets.append(abnormal_return)
            elif -0.05 <= abnormal_return < 0.05:
                self._targets.append(abnormal_return)
            elif abnormal_return >= 0.05:
                self._targets.append(abnormal_return)
            else:
                continue

            # Tracks how often words appear in the transcript
            tracker = defaultdict(int)

            for paragraph in TextGenerator(transcript):

                # Remembers last word for bigrams
                last_word = None

                for word in paragraph:

                    # Ignore stop words
                    if word in self.stop_words:
                        continue

                    # Ignore numbers
                    try:
                        float(word)
                        continue
                    except ValueError:
                        if last_word:
                            tracker[' '.join([last_word, word])] += 1
                        tracker[word] += 1
                        last_word = word

            for word in tracker:
                self._vocab[word][key] = tracker[word]

            if verbose:
                if key % 250 == 0:
                    print("PROGRESS: at transcript #{}, keeping {} word types".format(key, len(self._vocab)))

                if key % 1000 == 0:
                    self.trim_vocab(min_reduce=2)


    def TFIDF(self):

        print("Computing TF-IDF...",)
        # Compute inverse document frequency
        N = len(self._transcripts)
        idf = defaultdict(float)
        for word in self._vocab:
            freq = len(self._vocab[word])
            idf[word] = np.log(float(N) / freq)

        # Reaugment dictionary so transcripts are first-order keys and words are second-order keys
        # Also set self._vocab to now just be the actual vocabulary
        vocab = self._vocab.keys()
        self._features = {doc: {word: self._vocab[word][doc] for word in self._vocab if doc in self._vocab[word]}
                          for doc in self._transcripts}
        self._vocab = vocab

        for doc in self._features:
            max_freq = float(max(self._features[doc].values()))
            self._features[doc] = {word: ((count / max_freq) * 0.5 + 0.5) * idf[word] for word, count in
                                 self._features[doc].iteritems()}
        print("Done!")


    def trim_vocab(self, min_reduce):

        """ Trims all words that appear less than min_reduce times """

        start = len(self._vocab)
        for word in self._vocab.keys():
            if sum(self._vocab[word].values()) < min_reduce:
                del self._vocab[word]
        print("trimmed {} word types, {} word types remaining".format(start - len(self._vocab), len(self._vocab)))


    def finalize_vocab(self):

        while self.max_vocab_size and len(self._vocab) > self.max_vocab_size:
            self.trim_vocab(min_reduce=self.min_count)
            self.min_count += np.sqrt(np.sqrt(len(self.vocab)))

        print("collected {} word types".format(len(self._vocab)))


    def get_return(self, transcript):

        if self.target_period == 3:
            return transcript.return_3days
        elif self.target_period == 30:
            return transcript.return_30days
        elif self.target_period == 60:
            return transcript.return_60days
        else:
            return transcript.return_90days


    @property
    def transcripts(self):
        return self._transcripts

    @property
    def vocab(self):
        return self._vocab

    @property
    def inputs(self):
        return self._features

    @property
    def targets(self):
        return self._targets

In [4]:
Transcript = namedtuple('Transcript', ['company',       
                                       'ticker',        
                                       'date',          
                                       'return_3days',  
                                       'return_30days', 
                                       'return_60days', 
                                       'return_90days', 
                                       'prepared',      
                                       'QandA'])

In [5]:
def load_transcripts():                                            
    with gzip.open('../data/transcripts.p.gz') as f:         
        return pickle.load(f)                                      

In [ ]:
transcripts = load_transcripts()

In [ ]:
m = BiGramModel(transcripts, target_period=3, max_vocab_size=None, min_count=50)

PROGRESS: at transcript #250, keeping 273457 word types
PROGRESS: at transcript #500, keeping 477205 word types
PROGRESS: at transcript #750, keeping 645082 word types
PROGRESS: at transcript #1000, keeping 804645 word types
trimmed 579036 word types, 225609 word types remaining

In [ ]:
x = np.asarray([np.asarray([doc[word] if word in doc else 0 for word in m.vocab]) for doc in m.inputs.values()])
y = np.asarray(m.targets)

In [ ]:
print len(x), len(y)

In [ ]:
sss = StratifiedShuffleSplit(y, n_iter=1, test_size=0.25, random_state=0)
for train_index, test_index in sss:
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [ ]:
rf_clf = RandomForestClassifier(100)

In [ ]:
rf_clf.fit(x_train, y_train)
y_pred = rf_clf.predict(x_test)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
importances = zip(m.vocab, np.absolute(rf_clf.feature_importances_))
importances = sorted(importances, key=operator.itemgetter(1), reverse=True)

In [ ]:
sb.barplot(y=zip(*importances)[0][:25], x=zip(*importances)[1][:25], color='g', orient='h');

In [ ]:
lr_clf = LogisticRegression()

In [ ]:
lr_clf.fit(x_train, y_train)
y_pred = lr_clf.predict(x_test)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
importances = zip(m.vocab, np.sum(np.absolute(lr_clf.coef_), axis=0))
importances = sorted(importances, key=operator.itemgetter(1), reverse=True)

In [ ]:
sb.barplot(y=zip(*importances)[0][:25], x=zip(*importances)[1][:25], color='g', orient='h');